In [1]:
import matplotlib.pyplot as plt
import seaborn as sns; 
import pandas as pd
import numpy as np

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
from sklearn.linear_model import LogisticRegression
from sklearn.naive_bayes import GaussianNB
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
from sklearn.metrics import confusion_matrix
from sklearn.svm import SVC

sns.set(rc={'figure.figsize':(11.7,8.27)})

pd_csv = '../../data/pd_speech_features.csv'

In [2]:
def get_data(path):
  data = pd.read_csv(path, header=1)
  return data

def cor_heat(data, annot=False):
  corr = data.corr()
  sns.heatmap(corr, cmap='coolwarm_r', annot=annot)
  plt.show()

def split_data(data, scale=False, test_size=0.2):
  X = data.drop(data.columns[-1], axis=1)
  y = data[data.columns[-1]]

  if scale:
    scaler = StandardScaler()
    X = scaler.fit_transform(X)

  X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=test_size, random_state=387)
  return X_train, X_test, y_train, y_test

def scale(data):
  X = data.drop(data.columns[-1], axis=1)
  y = data[data.columns[-1]]

  scaler = StandardScaler()
  X = scaler.fit_transform(X)

  return pd.concat([X, y])

def hist(data, columns):

  if len(columns) > 1:
    _, ax = plt.subplots(1, len(columns), figsize=(30,10))
    for i in range(len(columns)):
      sns.histplot(data=data, x=columns[i], ax=ax[i])

  elif len(columns) == 1:
    sns.histplot(data=data, x=columns[0])

def evaluate_data(data):
  print('Null values:', data.isnull().sum().sum())
  pd.set_option('display.max_columns', None)
  display(data.describe())
  pd.reset_option('display.max_columns')

def remove_outliers(data, iqr_multiplier = 1.5, skip_columns = []):
  ret_data = data.copy()
  for column in ret_data.columns:
    if len(ret_data) == 0:
      break
    elif column in skip_columns:
      continue
    print(column)

    print('\tBefore removal:', len(ret_data))
    q25, q75 = np.percentile(ret_data, 25), np.percentile(ret_data, 75)
    iqr = q75 - q25

    cut_off = iqr * iqr_multiplier
    lower, upper = q25 - cut_off, q75 + cut_off

    ret_data =  ret_data.drop(ret_data[(ret_data[column] > upper) | (ret_data[column] < lower)].index)
    print('\tAfter removal: {}'.format(len(ret_data)))
  return ret_data

def remove_outliers_2(data, lower, upper):
  lower_percentile = data.quantile(lower)
  upper_percentile = data.quantile(upper)
  
  ret_data = data[(data >= lower_percentile) & (data <= upper_percentile)].dropna()
  
  return ret_data

def evaluate_model(y_test, y_pred, labels):
  # accuracy = accuracy_score(y_test, y_pred)
  # precision = precision_score(y_test, y_pred)
  # recall = recall_score(y_test, y_pred)
  # f1 = f1_score(y_test, y_pred)
  # roc_auc = roc_auc_score(y_test, y_pred)

  # print("Accuracy:", accuracy)
  # print("Precision:", precision)
  # print("Recall:", recall)
  # print("F1 Score:", f1)
  # print("ROC AUC Score:", roc_auc, '\n')

  # cr = classification_report(y_test, y_pred, target_names=labels, zero_division=0)
  # print(cr)
  
  cm = confusion_matrix(y_test, y_pred)

  true_positives = cm[1, 1]
  false_negatives = cm[1, 0]
  positive_recall = true_positives / (true_positives + false_negatives)
  print('Positive Recall:', positive_recall)
  
  # sns.heatmap(cm.T, square=True, annot=True, fmt='d', xticklabels = labels, yticklabels = labels)
  # plt.xlabel('true label')
  # plt.ylabel('predicted label')
  # plt.show()

def evaluate_models(data, models):
  # Preprocessing and Standardization
  X_train, X_test, y_train, y_test = split_data(data, scale=True)

  for model in models:
    # print(re.findall(r'\.(.*)', type(model)))

    print('\n\t~~~~\t', type(model).__name__, '\t~~~~\n')
    # Train the model
    model.fit(X_train, y_train)

    # Measure Model 
    y_pred = model.predict(X_test)

    # Evaluate the model's performance
    evaluate_model(y_test, y_pred, ['Negative', 'Positive'])

def get_important(data, top=20):
  X = data.drop(data.columns[-1], axis=1)
  y = data[data.columns[-1]]

  rf = RandomForestClassifier()
  rf.fit(X, y)

  feature_importances = rf.feature_importances_

  sorted_indices = np.argsort(feature_importances)[::-1]
  sorted_indices = sorted_indices[:top]
  
  selected_feature_names = X.columns[sorted_indices]
  selected_columns = np.concatenate([selected_feature_names, [data.columns[-1]]])
 
  selected_data = data[selected_columns]
  return selected_data

def get_pca(data, components):
  X = data.drop(data.columns[-1], axis=1)
  y = data[data.columns[-1]]

  pca = PCA(n_components=components)
  transformed_data = pca.fit_transform(X)

  transformed_df = pd.DataFrame(transformed_data, columns=[f"component_{i+1}" for i in range(components)])
  transformed_df['target'] = y.values

  return transformed_df


In [3]:
pd_data = get_data(pd_csv)
evaluate_data(pd_data)
print(pd_data['class'].value_counts())
print(round(pd_data['class'].value_counts()[1]/(pd_data['class'].value_counts()[0] + pd_data['class'].value_counts()[1] )* 100, 2), '% pd')
print('\n', pd_data.groupby('id')['class'].sum())


   id  gender      PPE      DFA     RPDE  numPulses  numPeriodsPulses  meanPeriodPulses  stdDevPeriodPulses  locPctJitter  locAbsJitter  rapJitter  ppq5Jitter  ddpJitter  locShimmer  locDbShimmer  apq3Shimmer  apq5Shimmer  apq11Shimmer  ddaShimmer  meanAutoCorrHarmonicity  meanNoiseToHarmHarmonicity  meanHarmToNoiseHarmonicity  minIntensity  maxIntensity  meanIntensity          f1           f2           f3           f4          b1          b2          b3          b4  GQ_prc5_95  GQ_std_cycle_open  GQ_std_cycle_closed  GNE_mean  GNE_std  GNE_SNR_TKEO  GNE_SNR_SEO  GNE_NSR_TKEO  GNE_NSR_SEO  VFER_mean  VFER_std  VFER_entropy  VFER_SNR_TKEO  VFER_SNR_SEO  VFER_NSR_TKEO  VFER_NSR_SEO  IMF_SNR_SEO  IMF_SNR_TKEO  IMF_SNR_entropy  IMF_NSR_SEO  IMF_NSR_TKEO  IMF_NSR_entropy  mean_Log_energy  mean_MFCC_0th_coef  mean_MFCC_1st_coef  mean_MFCC_2nd_coef  mean_MFCC_3rd_coef  mean_MFCC_4th_coef  mean_MFCC_5th_coef  mean_MFCC_6th_coef  mean_MFCC_7th_coef  mean_MFCC_8th_coef  mean_MFCC_9th_coef  mean_

,id,gender,PPE,DFA,RPDE,numPulses,numPeriodsPulses,meanPeriodPulses,stdDevPeriodPulses,locPctJitter,locAbsJitter,rapJitter,ppq5Jitter,ddpJitter,locShimmer,locDbShimmer,apq3Shimmer,apq5Shimmer,apq11Shimmer,ddaShimmer,meanAutoCorrHarmonicity,meanNoiseToHarmHarmonicity,meanHarmToNoiseHarmonicity,minIntensity,maxIntensity,meanIntensity,f1,f2,f3,f4,b1,b2,b3,b4,GQ_prc5_95,GQ_std_cycle_open,GQ_std_cycle_closed,GNE_mean,GNE_std,GNE_SNR_TKEO,GNE_SNR_SEO,GNE_NSR_TKEO,GNE_NSR_SEO,VFER_mean,VFER_std,VFER_entropy,VFER_SNR_TKEO,VFER_SNR_SEO,VFER_NSR_TKEO,VFER_NSR_SEO,IMF_SNR_SEO,IMF_SNR_TKEO,IMF_SNR_entropy,IMF_NSR_SEO,IMF_NSR_TKEO,IMF_NSR_entropy,mean_Log_energy,mean_MFCC_0th_coef,mean_MFCC_1st_coef,mean_MFCC_2nd_coef,mean_MFCC_3rd_coef,mean_MFCC_4th_coef,mean_MFCC_5th_coef,mean_MFCC_6th_coef,mean_MFCC_7th_coef,mean_MFCC_8th_coef,mean_MFCC_9th_coef,mean_MFCC_10th_coef,mean_MFCC_11th_coef,mean_MFCC_12th_coef,mean_delta_log_energy,mean_0th_delta,mean_1st_delta,mean_2nd_delta,mean_3rd_delta,mean_4th_delta,mean_5th_delta,mean_6th_delta,mean_7th_delta,mean_8th_delta,mean_9th_delta,mean_10th_delta,mean_11th_delta,mean_12th_delta,mean_delta_delta_log_energy,mean_delta_delta_0th,mean_1st_delta_delta,mean_2nd_delta_delta,mean_3rd_delta_delta,mean_4th_delta_delta,mean_5th_delta_delta,mean_6th_delta_delta,mean_7th_delta_delta,mean_8th_delta_delta,mean_9th_delta_delta,mean_10th_delta_delta,mean_11th_delta_delta,mean_12th_delta_delta,std_Log_energy,std_MFCC_0th_coef,std_MFCC_1st_coef,std_MFCC_2nd_coef,std_MFCC_3rd_coef,std_MFCC_4th_coef,std_MFCC_5th_coef,std_MFCC_6th_coef,std_MFCC_7th_coef,std_MFCC_8th_coef,std_MFCC_9th_coef,std_MFCC_10th_coef,std_MFCC_11th_coef,std_MFCC_12th_coef,std_delta_log_energy,std_0th_delta,std_1st_delta,std_2nd_delta,std_3rd_delta,std_4th_delta,std_5th_delta,std_6th_delta,std_7th_delta,std_8th_delta,std_9th_delta,std_10th_delta,std_11th_delta,std_12th_delta,std_delta_delta_log_energy,std_delta_delta_0th,std_1st_delta_delta,std_2nd_delta_delta,std_3rd_delta_delta,std_4th_delta_delta,std_5th_delta_delta,std_6th_delta_delta,std_7th_delta_delta,std_8th_delta_delta,std_9th_delta_delta,std_10th_delta_delta,std_11th_delta_delta,std_12th_delta_delta,Ea,Ed_1_coef,Ed_2_coef,Ed_3_coef,Ed_4_coef,Ed_5_coef,Ed_6_coef,Ed_7_coef,Ed_8_coef,Ed_9_coef,Ed_10_coef,det_entropy_shannon_1_coef,det_entropy_shannon_2_coef,det_entropy_shannon_3_coef,det_entropy_shannon_4_coef,det_entropy_shannon_5_coef,det_entropy_shannon_6_coef,det_entropy_shannon_7_coef,det_entropy_shannon_8_coef,det_entropy_shannon_9_coef,det_entropy_shannon_10_coef,det_entropy_log_1_coef,det_entropy_log_2_coef,det_entropy_log_3_coef,det_entropy_log_4_coef,det_entropy_log_5_coef,det_entropy_log_6_coef,det_entropy_log_7_coef,det_entropy_log_8_coef,det_entropy_log_9_coef,det_entropy_log_10_coef,det_TKEO_mean_1_coef,det_TKEO_mean_2_coef,det_TKEO_mean_3_coef,det_TKEO_mean_4_coef,det_TKEO_mean_5_coef,det_TKEO_mean_6_coef,det_TKEO_mean_7_coef,det_TKEO_mean_8_coef,det_TKEO_mean_9_coef,det_TKEO_mean_10_coef,det_TKEO_std_1_coef,det_TKEO_std_2_coef,det_TKEO_std_3_coef,det_TKEO_std_4_coef,det_TKEO_std_5_coef,det_TKEO_std_6_coef,det_TKEO_std_7_coef,det_TKEO_std_8_coef,det_TKEO_std_9_coef,det_TKEO_std_10_coef,app_entropy_shannon_1_coef,app_entropy_shannon_2_coef,app_entropy_shannon_3_coef,app_entropy_shannon_4_coef,app_entropy_shannon_5_coef,app_entropy_shannon_6_coef,app_entropy_shannon_7_coef,app_entropy_shannon_8_coef,app_entropy_shannon_9_coef,app_entropy_shannon_10_coef,app_entropy_log_1_coef,app_entropy_log_2_coef,app_entropy_log_3_coef,app_entropy_log_4_coef,app_entropy_log_5_coef,app_entropy_log_6_coef,app_entropy_log_7_coef,app_entropy_log_8_coef,app_entropy_log_9_coef,app_entropy_log_10_coef,app_det_TKEO_mean_1_coef,app_det_TKEO_mean_2_coef,app_det_TKEO_mean_3_coef,app_det_TKEO_mean_4_coef,app_det_TKEO_mean_5_coef,app_det_TKEO_mean_6_coef,app_det_TKEO_mean_7_coef,app_det_TKEO_mean_8_coef,app_det_TKEO_mean_9_coef,app_det_TKEO_mean_10_coef,app_TKEO_std_1_coef,app_TKEO_std_2_coef,app_TKEO_

class
1    564
0    192
Name: count, dtype: int64
74.6 % pd

 id
0      3
1      3
2      3
3      3
4      3
      ..
247    3
248    3
249    3
250    0
251    0
Name: class, Length: 252, dtype: int64


There seems to be 3 entries for each id value. Regardless this feature should be removed.<br>
There are also a lot of features with a lot of different deviations. The data should be normalized.

In [4]:
pd_data = get_data(pd_csv)
pd_data = pd_data.drop(['id'], axis=1)
remove_outliers(pd_data, skip_columns=[], iqr_multiplier=5)

gender
	Before removal: 756
	After removal: 756
PPE
	Before removal: 756
	After removal: 756
DFA
	Before removal: 756
	After removal: 756
RPDE
	Before removal: 756
	After removal: 756
numPulses
	Before removal: 756
	After removal: 1
numPeriodsPulses
	Before removal: 1
	After removal: 1
meanPeriodPulses
	Before removal: 1
	After removal: 1
stdDevPeriodPulses
	Before removal: 1
	After removal: 1
locPctJitter
	Before removal: 1
	After removal: 1
locAbsJitter
	Before removal: 1
	After removal: 1
rapJitter
	Before removal: 1
	After removal: 1
ppq5Jitter
	Before removal: 1
	After removal: 1
ddpJitter
	Before removal: 1
	After removal: 1
locShimmer
	Before removal: 1
	After removal: 1
locDbShimmer
	Before removal: 1
	After removal: 1
apq3Shimmer
	Before removal: 1
	After removal: 1
apq5Shimmer
	Before removal: 1
	After removal: 1
apq11Shimmer
	Before removal: 1
	After removal: 1
ddaShimmer
	Before removal: 1
	After removal: 1
meanAutoCorrHarmonicity
	Before removal: 1
	After removal: 1
meanNo

,gender,PPE,DFA,RPDE,numPulses,numPeriodsPulses,meanPeriodPulses,stdDevPeriodPulses,locPctJitter,locAbsJitter,...,tqwt_kurtosisValue_dec_28,tqwt_kurtosisValue_dec_29,tqwt_kurtosisValue_dec_30,tqwt_kurtosisValue_dec_31,tqwt_kurtosisValue_dec_32,tqwt_kurtosisValue_dec_33,tqwt_kurtosisValue_dec_34,tqwt_kurtosisValue_dec_35,tqwt_kurtosisValue_dec_36,class


In [5]:
pd_data = get_data(pd_csv)
pd_data = pd_data.drop(['id'], axis=1)

df = pd.DataFrame(pd_data)

normalized_df = (df - df.mean()) / df.std()

z_score_threshold = 20

outliers_mask = (normalized_df.abs() > z_score_threshold).any(axis=1)
filtered_df = df[~outliers_mask]

print("Original DataFrame:")
print(len(df))

print("\nFiltered DataFrame (without outliers):")
print(len(filtered_df))

Original DataFrame:
756

Filtered DataFrame (without outliers):
739


In [6]:

pd_data = get_data(pd_csv)

print('\n\n\n\t\t *** ALL Features ***\n\n\n')

evaluate_models(pd_data,
    [RandomForestClassifier(), 
     LogisticRegression(max_iter=1000),
     GradientBoostingClassifier(),
     SVC(),
     GaussianNB()])


for top in np.arange(50, 751, 50):
    print('\n\n\n\t\t *** TOP', top, 'Features ***\n\n')

    pd_data_top = get_important(pd_data, top)

    evaluate_models(pd_data_top,
        [RandomForestClassifier(), 
        LogisticRegression(max_iter=1000),
        GradientBoostingClassifier(),
        SVC(),
        GaussianNB()])
    
for components in np.arange(50, 751, 50):
    print('\n\n\n\t\t *** PCA', components, 'Components ***\n\n')

    pd_data_pcs = get_pca(pd_data, components)

    evaluate_models(pd_data_pcs,
        [RandomForestClassifier(), 
        LogisticRegression(max_iter=1000),
        GradientBoostingClassifier(),
        SVC(),
        GaussianNB()])
     




		 *** ALL Features ***




	~~~~	 RandomForestClassifier 	~~~~

Positive Recall: 0.9752066115702479

	~~~~	 LogisticRegression 	~~~~

Positive Recall: 0.8429752066115702

	~~~~	 GradientBoostingClassifier 	~~~~

Positive Recall: 0.9752066115702479

	~~~~	 SVC 	~~~~

Positive Recall: 0.9834710743801653

	~~~~	 GaussianNB 	~~~~

Positive Recall: 0.7520661157024794



		 *** TOP 50 Features ***



	~~~~	 RandomForestClassifier 	~~~~

Positive Recall: 0.9669421487603306

	~~~~	 LogisticRegression 	~~~~

Positive Recall: 0.9256198347107438

	~~~~	 GradientBoostingClassifier 	~~~~

Positive Recall: 0.9586776859504132

	~~~~	 SVC 	~~~~

Positive Recall: 0.9752066115702479

	~~~~	 GaussianNB 	~~~~

Positive Recall: 0.9090909090909091



		 *** TOP 100 Features ***



	~~~~	 RandomForestClassifier 	~~~~

Positive Recall: 0.9834710743801653

	~~~~	 LogisticRegression 	~~~~

Positive Recall: 0.9008264462809917

	~~~~	 GradientBoostingClassifier 	~~~~

Positive Recall: 0.9504132231404959

	~~~